In [2]:
import numpy as np
import os
import pandas as pd
import sys
import warnings
warnings.simplefilter('ignore')

#pkg_dir = '/home/mrossol/NaTGenPD'
pkg_dir = '..'
sys.path.append(pkg_dir)
import NaTGenPD as npd
import NaTGenPD.cluster as cluster


#data_dir = '/scratch/mrossol/CEMS'
data_dir = '/Users/mrossol/Downloads/CEMS'


def combine_raw_CEMS(raw_h5s):
    raw_df = []
    for raw_file in raw_h5s:
        raw_df.append(pd.read_hdf(raw_file, 'smoke_df'))
    
    return pd.concat(raw_df)


def get_process_stats(unit_fits, raw_df, clean_df, filtered_df):
    raw_units = list(unit_fits['unit_id'].values)
    pos = raw_df['unit_id'].isin(raw_units)
    raw_df = raw_df.loc[pos]
    raw_units = len(raw_df)
    raw_gen = 0
    total_points = 0
    non_zero_points = 0
    points_removed = 0
    non_zero_removed = 0
    for _, unit_df in raw_df.groupby('unit_id'):
        raw_gen += unit_df['load'].max()
        non_zero = (unit_df['load'] > 0).sum()
        raw_points = len(unit_df)
        non_zero_points += non_zero
        total_points += raw_points
        
    clean_units = 0
    clean_gen = 0
    for _, unit_df in clean_df.groupby('unit_id'):
        if not unit_df['load'].isnull().any():
            clean_units +=1
            clean_gen += unit_df['load'].nanmax()
            
    final_units = unit_fits['a0'].isnull()
    final_units = list(unit_fits.loc[final_units, 'unit_id'].values)
    filtered_units = 0
    filtered_gen = 0
    final_units = 0
    final_gen = 0
    final_points = 0
    for unit_id, unit_df in filtered_df.groupby('unit_id'):
        if not unit_df['load'].isnull().any():
            filtered_units += 1
            filtered_gen += unit_df['load'].nanmax()
        
        if unit_id in final_units:
            final_units += 1
            final_gen += unit_df['load'].nanmax()
            final_points += len(unit_df.loc[~unit_df['load'].isnull()])
    
    points_removed = raw_ponts - final_points
    non_zero_removed = non_zero - final_points
        
    
    out_stats = pd.Series({'raw_numb': raw_numb,
                           'raw_gen': raw_gen,
                           'clean_numb': clean_numb,
                           'clean_gen': clean_gen,
                           'filtered_numb': filter_numb,
                           'filtered_gen': filter_gen,
                           'final_numb': len(unit_fits),
                           'final_gen': unit_fits['load_max'].sum(),
                           'points_filtered': points_removed,
                           'non_zero_filtered': non_zero_removed,
                           'perc_filtered': points_removed / total_points,
                           'perc_non_zero_filtered': non_zero_removed / non_zero_points})
    return out_stats

# Procedure Stats

In [4]:
path = os.path.join(data_dir, 'Final_Fits')
final_fits = npd.Fits(path)

In [5]:
raw_paths = [os.path.join(data_dir, 'SMOKE_{}.h5'.format(y))
            for y in (2016, 2017)]
raw_df = combine_raw_CEMS(raw_paths)

ImportError: HDFStore requires PyTables, "No module named 'tables'" problem importing